# 09 추천시스템
## 08 파이썬 추천 시스템 패키지 - Surprise

### Surprise 패키지 소개

+ 파이썬 기반의 추천 시스템 구축을 위한 전용 패키지 중의 하나이다.
+ 파이썬 기반에서 사이킷런과 유사한 API와 프레임워크를 제공한다.
+ conda나 pip를 통해 설치한다.
  + pip install scikit-surprise
  + conda install -c conda-forge scikit-surprise

+ Surprise 패키지의 장점
  + 다양한 추천 알고리즘, 예를 들어 사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링을 쉽게 적용해 추천 시스템을 구축할 수 있다.
  + Surprise의 핵심 API는 사이킷런의 핵심 API와 유사한 API명으로 작성됐다. 
    + fit(), predict() API로 추천 데이터 학습과 예측
    + train_test_split()으로 추천 학습 데이터 세트와 예측 데이터 세트 분리
    + cross_valdate(),  GridSearchCV 클래스를 통해 추천 시스템을 위한 모델 셀렉션, 평가, 하이퍼 파라미터 튜닝 등의 기능을 제공한다.

### Surprise를 이용한 추천 시스템 구축

+ 예제는 추천 데이터를 학습용과 테스트용 데이터 세트로 분리한 뒤 SVD 행렬 분해를 통한 잠재 요인 협업 필터링을 수행한다.

In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

+ 데이터 세트 로딩
  + Surprise에서 데이터 로딩은 Dataset 클래스를 이용해서만 가능하다.
  + Surprise는 userID(사용자 ID), movieID(영화 ID), rating(평점)과 같은 주요 데이터가 로우(Row) 레벨 형태로 돼있는 포맷의 데이터만 처리한다.

Surprise Dataset 클래스의 load_bulletin() 은 무비렌즈 사이트에서 제공하는 과거 버전 데이터 세트인 'ml-100k' 또는 'ml-1m' 데이터를 아카이브 사이트로부터 내려받아 로컬 디렉터리에 저장한 뒤 데이터를 로딩한다. 로딩한 데이터를 Surprise 패키지의 train_test_split() API를 이용해 학습 데이터 세트와 테스트 데이터 세트로 분리해 본다.

In [2]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\구남이/.surprise_data/ml-100k


Surprise가 내려받은 ml-100k, ml-1m은 과거 버전의 데이터 세트이기 때문에 분리문자가 탭(\t) 문자이다.

Surprise에 사용자-아이템 평점 데이터를 적용할 때 주의해야할 점은 무비렌즈 사이트에서 내렵다은 데이터 파일과 동일하게 로우 레벨의 사용자-아이템 평점 데이터를 그대로 적용해야 한다는 것이다. Surprise는 자체적으로 로우 레벨의 데이터를 칼럼 레벨의 데이터로 변경하므로 원본인 로우 레벨의 사용자-아이템 평점 데이터를 데이터 세트로 적용해야 한다.

이제 SVD로 잠재 요인 협업 필터링을 수행한다. 적용하는 데이터 세트는 앞에서 소개한 train_test_split()으로 분리된 학습 데이터 세트이다. 

먼저 algo = SVD()와 같이 알고리즘 객체를 생성한다. 이 알고리즘 객체에 fit(학습 데이터 세트)을 수행해 학습 데이터 세트 기반으로 추천 알고리즘을 학습한다.

In [3]:
algo = SVD(random_state=0)
algo.fit(trainset) 

학습된 추천 알고리즘을 기반으로 테스트 데이터 세트에 대해 추천을 수행한다. Surprise에서 추천을 예측하는 메서드는 test()와 predict() 두개이다.

+ test()
  + 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드이다. 
  + 즉, 입력된 데이터 세트에 대해 추천 데이터 세트를 만들어준다.
+ predict()
  + 개별 사용자와 영화에 대한 추천 평점을 반환해준다.

In [4]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

SVD 알고리즘 객체의 test(데이터 세트) 메서드의 호출 결과는 파이썬 리스트이며 크기는 입력 인자 데이터 세트의 크기와 같은 25,000개이다. 호출 결과로 반환된 리스트 객체는 25,000개의 Prediction 객체를 내부에 가지고 있다. Prediction 객체는 Surprise 패키지에서 제공하는 데이터 타입이며, 개별 사용자 아이디(uid), 영화(또는 아이템) 아이디(iid)와 실제 평점(r_ui) 정보에 기반해 Surprise의추천 예측 평점(est) 데이터를 튜플 형태로 가지고 있다. 

Prediction 객체의 details 속성은 내부 처리 시 추천 예측을 할 수 없는 경우에 로그용으로 데이터를 남기는데 사용된다.

'was_impossible'이 True이면 예측값을 생성할 수 없는 데이터라는 의미이다. 여기서는 모두 False이다.

리스트 객체 내에 내포된 Prediction 객체의 uid, iid, r_ui, est 등의 속성에 접근하려면 객체명.uid와 같은 형식으로 가능하다.

In [5]:
# Prediction 객체에서 uid, iid, est 속성을 추출
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

이번에는 Surprise 패키지의 다른 추천 예측 메서드인 predict()를 이용해 추천 예측을 해본다. 
+ predict()
  + 개별 사용자의 아이템에 대한 추천 평점을 예측해준다.
  + 인자로 개별 사용자 아이디, 아이템 아이디를 입력하면 추천 예측 평점을 포함한 정보를 반환한다.

In [6]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


결과처럼 predict()는 개별 사용자와 아이템 정보를 입력하면 추천 예측 평점을 est로 반환한다.

테스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점과의 차이를 평가해본다.

Surprise의 accuracy 모듈은 RMSE, MSE 등의 방법으로 추천 시스템의 성능 평가 정보를 제공한다. accuracy모듈의 rmse()를 이용해 RMSE 평과 결과를 확인해본다.

In [7]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

### Surpirse 주요 모듈 소개

__Dataset__

+ Surprise는 user_id (사용자 아이디), item_id(아이템 아이디), rating(평점) 데이터가 로우 레벨로 된 데이터 세트만 적용할 수 있다.
+ 데이터의 첫 번째 칼럼을 사용자 아이디, 두번째 칼럼을 아이템 아이디, 세 번째 칼럼을 평점으로 가정해 데이터를 로딩하고 네 번째 칼럼부터는 아예 로딩을 수행하지 않는다.
+ 무비렌즈 아카이브 서버에서 자동으로 내려받는 데이터 파일뿐만 아니라 일반 데이터 파일이나 판다스 DataFrame에서도 로딩할 수 있다. 
  + 단, 데이터 세트의 칼럼 순서가 사용자 아이디, 아이템 아이디, 평점 순으로 반드시 돼 있어야 한다.

API 명 | 내용
-------|------
Dataset.load_builtin(name='ml-100k') | 무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터를 내려받는다. </br> ml-100k , ml-1M를 내려받을 수 있다. 일단 내려받은 데이터는 .surprise_data 디렉터리 밑에 저장되고, 해당 디렉터리에 데이터가 있으면 FTP에서 내려받지 않고 해당 데이터를 이용한다. 입력 파라미터인 name으로 대상 데이터가 ml-100k인지 ml-1m인지를 입력한다. (name='ml-100k'). 디폴트는 ml-100k이다.
Dataset.load_from_file(fil_path, reader) | OS 파일에서 데이터를 로딩할 때 사용한다. </br> 콤마, 탭 등으로 칼럼이 분리된 포맷의 OS 파일에서 데이터를 로딩한다. </br> 입력 파라미터로 OS 파일명, Reader로 파일의 포맷을 지정한다.
Dataset.load_from_df(df, reader) | 판다스의 DataFrame에서 데이터를 로딩한다. </br> 파라미터로 DataFrame을 입력받으며 DataFrame 역시 반드시 3개의 칼럼인 사용자 아이디, 아이템 아이디, 평점 순으로 칼럼 순서가 정해져 있어야 한다. </br> 입력 파라미터로 DataFrame 객체, Reader로 파일의 포맷을 지정한다.

__OS 파일 데이터를 Surprise 데이터 세트로 로딩__

Dataset.load_from_file API를 이용해 지정된 디렉터리에 있는 사용자-아이템 평점 데이터를 로딩한다. 이제 부터 사용하는 데이터파일은 ratings.csv와 movies.csv이다.

Surprise에 OS 파일을 로딩할 때의 주의할 점은 로딩되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있어서는 안된다는 것이다. 여기서 사용할 ratings.csv 파일은 맨 처음 위치에 칼럼명을 헤더로 가지고 있으므로 판다스 DataFrame의 to_csv() 함수를 이용해 간단하게 이 칼럼헤덜르 삭제하고 새로운 파일인 ratings_noh.csv로 저장한다.

In [8]:
import pandas as pd

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index=False, header=False)

이제 ratings_noh.csv를 DataSet 모듈의 load_from_file()을 이용해 DataSet로 로드한다. 먼저 Dataset.load_from_file()을 적용하기 전에 Reader 클래스를 이용해 데이터 파일의 파싱 포맷을 정의해야 한다. Reader 클래스는 로딩될 ratings_noh.csv 파일의 파싱 정보를 알려주기 위해 사용된다. 

지금 로딩하려는 ratings_noh.csv는 칼럼 헤더가 없고, 4개의 칼럼이 콤마로만 분리돼 있다. 이 4개의 칼럼이 사용자 아이디, 아이템 아이디, 평점, 타임스탬프임을 로딩할 때 알려줘야 한다.

Reader 클래스의 생성자에 각 필드의 칼럼명과 칼럼 분리문자, 그리고 최소~최대 평점을 입력해 객체를 생성하고, load_from_file()로 생성된 Reader 객체를 참조해 데이터 파일을 파싱하면서 로딩한다.

In [9]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))

이렇게 Reader 설정이 완료되면 Dataset.load_from_file()은 이를 기반으로 데이터를 파싱하면서 Dataset를 로딩한다. 로딩 시 ratings_noh.csv 파일에서 앞의 3개 칼럼만 로딩되고 timestamp 칼럼은 제외된다.

In [10]:
data=Dataset.load_from_file('./ml-latest-small/ratings_noh.csv',reader=reader)

+ Reader 클래스의 주요 생성 파라미터
  + line_format (string) : 칼럼을 순서대로 나열한다. 입력된 문자열을 공백으로 분리해 칼럼으로 인식한다.
  + sep (char) : 칼럼을 분리하는 분리자이며, 디폴트는 '\t'이다. 판다스 DataFrame 에서 입력받을 경우에는 기재할 필요가 없다.
  + rating_scale (tuple, optional) : 평점값의 최소 ~ 최대 평점을 설정한다. 디폴트는 (1,5)이지만 ratings.csv 파일의 경우는 최소 평점이 0.5, 최대 평점이 5이므로 (0.5, 5)로 설정한다.

이제 SVD 행렬 분해 기법을 이용해 추천을 예측한다. 잠재 요인 크기 K값을 나타내는 파라미터인 n_factors를 50으로 설정해 데이터를 학습한 뒤에 테스트 데이터 세트를 적용해 예측 평점을 구한다. 그리고 예측 평점과 실제 평점 데이터를 RMSE로 평가한다.

In [11]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

__판다스 DataFrame에서 Surprise 데이터 세트로 로딩__

Dataset.load_from_df()를 이용하면 판다스의 DataFrame에서도 Surprise 데이터 세트로 로딩할 수 있다. 주의할 점은 DataFrame 역시 사용자 아이디, 아이템 아이디, 평점 칼럼 순서를 지켜야 한다는 것이다.

ratings.csv 파일을 DataFrmae으로 로딩한 ratings에서 Surprise 데이터 세트로 로딩하려면 Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader) 와 같이 파라미터를 입력하면 된다. 

다음은 이를 이용한 SVD 추천 예측을 코드로 작성한 것이다.

In [12]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./ml-latest-small/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### Surprise 추천 알고리즘 클래스

클래스명 | 설명
---------|-------
SVD | 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
KNNBasic | 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
BaselineOnly | 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘

이 밖에도 SVD++, NMF 등 다양한 유형의 알고리즘을 수행할 수 있다. 지원 알고리즘은 https://surprise.readthedocs.io/en/stable/prediction_algorithms_pckage.html 에서 참조할 수 있다.

+ SVD 클래스의 입력 파라미터
    + 주로 n_factors와 n_epochs의 값을 변경해 튜닝할 수 있으나 튜닝 효과는 크지 않다. biased의 경우는 큰 이슈가 없는 한 디폴트인 True로 설정을 유지하는 것이 좋다.

파라미터명 | 내용
----------|----------
n_factors | 잠재 요인 K의 개수, 디폴트는 100, 커질수록 정확도가 높아질 수 있으나 과적합 문제가 발생할 수 있다.
n_epochs | SGD(Stochastic Gradient Descent) 수행 시 반복 횟수, 디폴트는 20
biased (bool) | 베이스라인 사용자 편향 적용 여부이며 디폴트는 True이다.

+ 베이스라인 평점
  + 개인의 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영하여 평점을 부과하는 것을 베이스라인 평점(Baseline Rating)이라고 한다.
  + 보통 베이스라인 평점은 (전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수) 공식으로 계산된다.
    + 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
    + 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
    + 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점

### 교차 검증과 하이퍼 파라미터 튜닝

Surprise는 교차 검증과 하이퍼 파라미터 튜닝을 위해 사이킷런과 유사한 cross_validate()와 GridSearchCV 클래스를 제공한다.

+ cross_validate()
  + 해당 함수는 surprise.model_selectoin 모듈 내에 존재하며, 폴드된 데이터 세트의 개수와 성능 측정 방법을 명시해 교차 검증을 수행한다.
  + 해당 함수의 인자로는 알고리즘 객체, 데이터, 성능 평가 방법(measures), 폴드 데이터 세트 개수(cv)를 입력한다.

다음 예제에서는 cross_validate()를 이용해 ratings.csv를 DataFrame으로 로딩한 데이터를 5개의 학습/검증 폴드 데이터 세트로 분리해 교차 검증을 수행하고 RMSE, MAE로 성능 평가를 진행한다.

In [13]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('./ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8728  0.8758  0.8734  0.8745  0.8713  0.8736  0.0015  
MAE (testset)     0.6699  0.6734  0.6709  0.6718  0.6698  0.6712  0.0013  
Fit time          16.69   14.65   15.97   15.68   17.32   16.06   0.91    
Test time         0.47    0.65    0.55    0.41    0.91    0.60    0.18    


{'test_rmse': array([0.87283632, 0.87579619, 0.87340892, 0.87447148, 0.87130101]),
 'test_mae': array([0.66991708, 0.67342932, 0.67091637, 0.67179506, 0.66979866]),
 'fit_time': (16.691898107528687,
  14.650868654251099,
  15.97023606300354,
  15.684734582901001,
  17.31791615486145),
 'test_time': (0.471738338470459,
  0.6542510986328125,
  0.5505425930023193,
  0.4100453853607178,
  0.9135572910308838)}

cross_validate()는 위의 출력 결과와 같이 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 함께 보여준다. Surprise의 GridSearchCV도 사이킷런의 GridSearchCV와 유사하게 교차 검증을 통한 하이퍼 파라미터 최적화를 수행한다. 

SVD의 경우 주로 점진적 하강 방식(Stochastic Gradient Descent)의 반복횟수를 지정하는 n_epochs와 SVD 잠재 요인K의 크기를 지정하는 n_factors를 튜닝한다.

n_epochs와 n_factors를 변경하면서 CV가 3일 때의 최적 하이퍼 파라미터를 도출해본다.

In [14]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8773813607328308
{'n_epochs': 20, 'n_factors': 50}


3개 폴드의 검증 데이터 세트에서 최적 RMSE가 약 0.8769로 도출됐다.

### Surprise를 이용한 개인화 영화 추천 시스템 구축

Surprise를 이용해 잠재 요인 협업 필터링 기반의 개인화된 영화 추천을 구현해본다. 이제는 Surprise 패키지로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은(관람하지 않은) 영화 중에서 개인 취향에 가장 적절한 영화를 추천해본다.

이번에는 ratings.csv를 학습 데이터와 테스트 데이터로 분리하지 않고 전체를 학습데이터로 사용한다. 그런데 Surprise는 데이터 세트를 train_test_split()을 이용해 내부에서 사용하는 TrainSet 클래스 객체로 변환하지 않으면 fit()을 통해 학습할 수가 없다.

데이터 세트 전체를 학습 데이터로 사용하려면 DatasetAutoFolds 클래스를 이용하면 된다. DatasetAutoFolds 객체를 생성한 뒤에 build_full_trainset() 메서드를 호출하면 전체 데이터를 학습 데이터 세트로 만들 수 있다.

In [15]:

from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

생성된 학습 데이터를 기반을 학습을 수행한다. 그리고 이후에 특정 사용자에게 영화를 추천하기 위해 아직 보지 않은 영화목록을 확인해본다.

먼저 SVD를 이용해 학습을 수행한다.

In [16]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

특정 사용자는 userID=9인 사용자로 지정한다. userId 9가 아직 평점을 매기지 않은 영화를 movieId 42로 선정한 뒤 예측 평점을 계산해 본다. 영화의 상세 정보는 movies.csv에 있으므로 해당 파일을 DataFrame으로 로딩한다.

In [17]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


이 movieId 42 영화에 대해 userId 9 사용자의 추천 예상 평점은 predict() 메서드를 이용하면 알 수 있다. 학습된 SVD객체에서 predict() 메서드 내에 userId와 moviedId 값을 입력해주면 된다. 단, 이 값은 모두 문자열 값이여야한다.

In [18]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


추천 예측 평점은 est 값으로 3.13이다.

이제 사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤에 예측 평점 순으로 영화를 추천해본다.

새롭게 get_unseen_surprise() 함수를 만들고 이를 이용해 아이디 9인 사용자가 아직 평점을 매기지 않은 영화 정보를 반환한다.

In [19]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


사용자 아이디 9번은 전체 9742개의 영화 중에서 46개만 평점을 매겼다. 따라서 추천 대상 영화는 9696개이며, 이중 앞에서 학습된 추천 알고리즘 클래스인 SVD를 이용해 높은 예측 평점을 가진 순으로 영화를 추천해본다.

recomm_movie_by_surprise() 함수를 새롭게 생성한다. recomm_movie_by_surprise()는 추천 대상 영화 모두를 대상으로 추천 알고리즘 객체의 predict() 메서드를 호출하고 그 결과인 Prediction 객체를 리스트 객체로 저장한다. 그리고 이렇게 저장된 리스트 내부의 Prediction 객체의 예측 평점이 높은 순으로 다시 정렬한 뒤 Top-N개의 Prediction 객체에서 영화 아이디, 영화 제목, 예측 평점 정보를 추출해 반환한다.

In [20]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


9번 아이디 사용자에게느느 케빈 스페이시 주연의 '유주얼 서스펙트', 그리고 '펄프픽션', '양들의 침묵', '대부'와 '좋은 친구들' 같은 서스펜스/스릴러/범죄 영화 및 스타워즈와 같은 액션 영화 등이 주로 추천됐다.